# Initializations #

In [40]:
import numpy as np
import pandas as pd

data = pd.read_csv("Airplane.csv",nrows=10000)
data.head()

Unnamed: 0      id  Gender      Customer Type  Age   Type of Travel  \
0           0   70172    Male     Loyal Customer   13  Personal Travel   
1           1    5047    Male  disloyal Customer   25  Business travel   
2           2  110028  Female     Loyal Customer   26  Business travel   
3           3   24026  Female     Loyal Customer   25  Business travel   
4           4  119299    Male     Loyal Customer   61  Business travel   

      Class  Flight Distance  Inflight wifi service  \
0  Eco Plus              460                      3   
1  Business              235                      3   
2  Business             1142                      2   
3  Business              562                      2   
4  Business              214                      3   

   Departure/Arrival time convenient  ...  Inflight entertainment  \
0                                  4  ...                       5   
1                                  2  ...                       1   
2                                  2  ...                       5   
3                                  5  ...                       2   
4                                  3  ...                       3   

   On-board service  Leg room service  Baggage handling  Checkin service  \
0                 4                 3                 4                4   
1                 1                 5                 3                1   
2                 4                 3                 4                4   
3                 2                 5                 3                1   
4                 3                 4                 4                3   

   Inflight service  Cleanliness  Departure Delay in Minutes  \
0                 5            5                          25   
1                 4            1                           1   
2                 4            5                           0   
3                 4            2                          11   
4                 3            3                           0   

   Arrival Delay in Minutes             satisfaction  
0                      18.0  neutral or dissatisfied  
1                       6.0  neutral or dissatisfied  
2                       0.0                satisfied  
3                       9.0  neutral or dissatisfied  
4                       0.0                satisfied  

[5 rows x 25 columns]

# Mappings #

In [41]:
data["Customer Type"] = data["Customer Type"].map({"Loyal Customer": 0, "disloyal Customer": 1}).astype(int)
data["Type of Travel"] = data["Type of Travel"].map({"Personal Travel": 0, "Business travel": 1}).astype(int)
data["Class"] = data["Class"].map({"Eco Plus": 0, "Business": 1, "Eco": 2}).astype(int)
data['Gender'] = data['Gender'].map({"Male": 1, "Female": 0}).astype(int)


data.loc[data['Arrival Delay in Minutes'] <= 5, 'Arrival Delay in Minutes'] = 0
data.loc[(data['Arrival Delay in Minutes'] > 5) & (data['Arrival Delay in Minutes'] <= 15), 'Arrival Delay in Minutes'] = 1
data.loc[(data['Arrival Delay in Minutes'] > 15) & (data['Arrival Delay in Minutes'] <= 30), 'Arrival Delay in Minutes'] = 2
data.loc[data['Arrival Delay in Minutes'] > 30, 'Arrival Delay in Minutes'] = 3
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].fillna(0).astype(int)
 
data.loc[data['Age'] <= 15, 'Age'] = 0
data.loc[(data['Age'] > 15) & (data['Age'] <= 25), 'Age'] = 1
data.loc[(data['Age'] > 25) & (data['Age'] <= 40), 'Age'] = 2
data.loc[data['Age'] > 40, 'Age'] = 3

data.loc[data['Flight Distance'] <= 200, 'Flight Distance'] = 0
data.loc[(data['Flight Distance'] > 200) & (data['Flight Distance'] <= 500), 'Flight Distance'] = 1
data.loc[(data['Flight Distance'] > 500) & (data['Flight Distance'] <= 1000), 'Flight Distance'] = 2
data.loc[data['Flight Distance'] > 1000, 'Flight Distance'] = 3
 
data.loc[data['Departure Delay in Minutes'] <= 5, 'Departure Delay in Minutes'] = 0
data.loc[(data['Departure Delay in Minutes'] > 5) & (data['Age'] <= 15), 'Departure Delay in Minutes'] = 1
data.loc[(data['Departure Delay in Minutes'] > 15) & (data['Age'] <= 30), 'Departure Delay in Minutes'] = 2
data.loc[data['Departure Delay in Minutes'] > 30, 'Departure Delay in Minutes'] = 3 

data["satisfaction"] = data["satisfaction"].map({"neutral or dissatisfied": 0, "satisfied": 1}).astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Unnamed: 0                         10000 non-null  int64
 1   id                                 10000 non-null  int64
 2   Gender                             10000 non-null  int32
 3   Customer Type                      10000 non-null  int32
 4   Age                                10000 non-null  int64
 5   Type of Travel                     10000 non-null  int32
 6   Class                              10000 non-null  int32
 7   Flight Distance                    10000 non-null  int64
 8   Inflight wifi service              10000 non-null  int64
 9   Departure/Arrival time convenient  10000 non-null  int64
 10  Ease of Online booking             10000 non-null  int64
 11  Gate location                      10000 non-null  int64
 12  Food and drink     

# Testing #

In [42]:
from DecisionTree import DecisionTree
import entropy
import gini_index
import calculations as cl
from sklearn.model_selection import train_test_split

_data = data.drop(["id", "Unnamed: 0","satisfaction"], axis=1).values
target = data["satisfaction"].values

# 80 to 20 ratio
train_data, test_data, train_target, test_target = train_test_split(
    _data, target, test_size=0.2, random_state=1234
)

gini_tree = DecisionTree(max_depth=50, impurity_function=gini_index.calculate)
entropy_tree = DecisionTree(max_depth=50, impurity_function=entropy.calculate)

gini_tree.fit(train_data, train_target)
entropy_tree.fit(train_data, train_target)

gini_predictions = entropy_tree.predict(test_data)
entropy_predictions = gini_tree.predict(test_data)

print(f'Gini Tree Accuracy: {cl.accuracy(test_target, gini_predictions) * 100}')
print(f'Entropy Tree Accuracy: {cl.accuracy(test_target, entropy_predictions) * 100}')

Gini Tree Accuracy: 93.8
Entropy Tree Accuracy: 92.95


# Entropy Tree Plot #

In [43]:
entropy_tree.print_tree(entropy_tree.root)

feature: 11, threshold: 3, value: None
left-> 	feature: 6, threshold: 0, value: None
right-> 	feature: 3, threshold: 0, value: None


feature: 6, threshold: 0, value: None
left-> 	feature: None, threshold: None, value: 1
right-> 	feature: 6, threshold: 3, value: None


feature: None, threshold: None, value: 1
left-> 	right-> 	

feature: 6, threshold: 3, value: None
left-> 	feature: 3, threshold: 0, value: None
right-> 	feature: 6, threshold: 4, value: None


feature: 3, threshold: 0, value: None
left-> 	feature: None, threshold: None, value: 0
right-> 	feature: 1, threshold: 0, value: None


feature: None, threshold: None, value: 0
left-> 	right-> 	

feature: 1, threshold: 0, value: None
left-> 	feature: 13, threshold: 3, value: None
right-> 	feature: 7, threshold: 1, value: None


feature: 13, threshold: 3, value: None
left-> 	feature: 17, threshold: 4, value: None
right-> 	feature: 16, threshold: 3, value: None


feature: 17, threshold: 4, value: None
left-> 	feature: 16, threshold: 

# Gini Tree Plot #

In [44]:
gini_tree.print_tree(gini_tree.root)

feature: 11, threshold: 3, value: None
left-> 	feature: 6, threshold: 0, value: None
right-> 	feature: 3, threshold: 0, value: None


feature: 6, threshold: 0, value: None
left-> 	feature: None, threshold: None, value: 1
right-> 	feature: 6, threshold: 3, value: None


feature: None, threshold: None, value: 1
left-> 	right-> 	

feature: 6, threshold: 3, value: None
left-> 	feature: 4, threshold: 1, value: None
right-> 	feature: 6, threshold: 4, value: None


feature: 4, threshold: 1, value: None
left-> 	feature: 13, threshold: 3, value: None
right-> 	feature: 8, threshold: 3, value: None


feature: 13, threshold: 3, value: None
left-> 	feature: 12, threshold: 4, value: None
right-> 	feature: 19, threshold: 3, value: None


feature: 12, threshold: 4, value: None
left-> 	feature: 19, threshold: 4, value: None
right-> 	feature: 3, threshold: 0, value: None


feature: 19, threshold: 4, value: None
left-> 	feature: 7, threshold: 3, value: None
right-> 	feature: 13, threshold: 1, value: None